# Multi-Agent Stock Analysis

In [1]:
!pip install pyautogen groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.2 MB/s eta 0:00:00


In [2]:
## import required libraries
import os
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen import ConversableAgent
from google.colab import userdata
from groq import Groq

In [3]:
## llm configuration
llm_config ={
                "model": "llama-3.3-70b-versatile",
                "api_key": userdata.get("GROQ_API_KEY"),
                "api_type": "groq"
            }

## The task!

In [4]:
task = "Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief report."

## Create a group chat

This group chat will include these agents:

- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Engineer: to write code using the defined plan by the planner.
- Executor: to execute the code written by the engineer.
- Writer: to write the report.

In [5]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="Give the task, and send instructions to the writer to refine the stock analysis report.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

In [6]:
## planner agent
planner = ConversableAgent(
    name="Planner",
    system_message="""
    Given a task, please determine what stock information is needed to complete the task.
    Please note that the information will all be retrieved using Python code.
    Please only suggest information that can be retrieved using Python code.
    After each step is done by others, check the progress and instruct the remaining steps.
    If a step fails, try to workaround.
    """,
    llm_config=llm_config,
)

In [7]:
## engineer agent
engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code to fetch and analyze stock data based on the plan provided by the planner.",
)

In [8]:
## executor agent
executor = ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the engineer and report the stock data results.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 6,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [9]:
## writer agent
writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Stock Analysis Report Writer. Please write stock analysis reports in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You take feedback from the admin and refine your report.",
    description="Writer. Write stock analysis reports based on the code execution results and take feedback from the admin to refine the report."
)

## Add a speaker selection policy

In [10]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [11]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

## Start the group chat

In [12]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief report.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...


/usr/local/lib/python3.11/dist-packages/autogen/oai/groq.py:303: UserWarning: Cost calculation not available for model llama-3.3-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Planner (to chat_manager):

**Step 1: Retrieve historical daily closing prices of Apple (AAPL) stock for the past 1 month**

To complete this task, we will need the following information:

* Historical daily closing prices of AAPL stock
* Date range: past 1 month

We can use the `yfinance` library in Python to retrieve this information.

**Python Code:**
```python
import yfinance as yf
from datetime import datetime, timedelta

# Define the ticker symbol and date range
ticker = 'AAPL'
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')

# Retrieve historical daily closing prices
data = yf.download(ticker, start=start_date, end=end_date)

# Extract closing prices
closing_prices = data['Close']
```

**Next Steps:**

Please execute the above code and store the `closing_prices` data. Once this step is done, I will instruct the next steps to analyze the data and create a brief report.

Please confirm once this step is com